In [6]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

### Optimization Problems

In [17]:
def solve_master():

    # model
    m = gp.Model()

    # variables + objective function
    z = m.addVars(i, q, obj = f, vtype = GRB.BINARY, name = "z")
    eta = m.addVar(1, obj = 1)

    # model sense
    m.ModelSense = GRB.MINIMIZE

    # constraints
    m.addConstr(gp.quicksum(gp.quicksum(b[i, q] * z[i, q] for i in H) for q in Q) >= gp.quicksum(W[k] for k in K))
    m.addConstrs(gp.quicksum(z[i, q]) <= 1 for i in H)

    # update and solve the model
    m.update()
    m.optimize()

    # optimal solution
    for var in m.getVars():
        print(f"{var.VarName} = {var.x}")

    return None

In [16]:
def solve_dual_subproblem():

    # model
    m = gp.Model()

    # variables + objective function
    alpha = m.addVars(k, obj = 1, name = "alpha")
    u = m.addVars(i, k, obj = z_hat, lb = 0, name = "u")
    v = m.addVars(i, obj = b * z_hat, lb = 0, name = "v")

    # model sense
    m.ModelSense = GRB.MAXIMIZE

    # constraints
    m.addConstrs(alpha[k] - u[a1][k] - u[a2][k] - W[k] * v[a1] <= F[a][k] for k in K for a in A_k_2)
    m.addConstrs(alpha[k] - u[a1][k] - W[k] * v[a1] <= F[a][k] for k in K for a in A_k_1)

    # update and solve the model
    m.update()
    m.optimize()

    # optimal solution
    for var in m.getVars():
        print(f"{var.VarName} = {var.x}")


    return None

### Algorithms

In [9]:
def benders_decomp():
    upper_bound = 0
    t = 0
    dual_extreme_pts = set()
    terminate = False

    while terminate == False:
        phi = solver_master() # solve the master problem and obtain the optimal objective function value

        if phi == upper_bound:
            terminate = True
        else:
            dual_soln = solve_dual_subproblem()
            dual_extreme_pts.add(dual_soln)

            if opt_dual_value + ... < upper_bound:
                upper_bound = opt_dual_value + ...

        t += 1

    return None

In [12]:
def exact_nmchlp():
    # Solve a problem
    # Solve a problem


    ub = min(min, phi, np.inf)

    for i, q in zip(H(S*), Q_i(S*)):
        z_iq = 0
        # Solve problem

        if phi(MP^T) > UB:
            z_iq = 1
        

    for i, q in zip(H*, Q_i*):
        z_iq = 1
        # Solve MP^T

        if phi(MP^T) > UB:
            Q_i* = Q_i*.pop(q)

            if len(Q_i*) == 0:
                H* = H*.pop(i)

    S = set((i, q) for i, q in zip(H*, Q_i*))

    for S' subset S s.t. (19), (20) hold:
        # Sovle GAP problem

        if phi(GAP(z(S'))) < UB:
            UB = phi(GAP(z(S')))
                           
    return None